In [1]:
include("../EDKit.jl")
using Main.EDKit, LinearAlgebra, Test, Profile

## Eigenvalue Test

In [2]:
a = 1.0 
b = randn()
c = randn()
d = randn()
e = randn()
f = randn()

mat = spin(
    (a, "xx11"), (a, "yy11"), 
    (b, "x1x1"), (b, "y1y1"), 
    (c, "x11x"), (c, "y11y"), 
    (d, "zz11"),
    (e, "z1z1"), 
    (f, "z11z")
)

function add_value!(E, B)
    iszero(size(B, 1)) && return 
    vals = trans_inv_operator(mat, 4, B) |> Hermitian |> eigvals
    append!(E, vals)
end

@testset "1-sym-vals" begin
    for L in 4:10 
        B = basis(;L)
        E = trans_inv_operator(mat, 4, B) |> Hermitian |> eigvals
        
        # N total 
        Es = []
        for N in 0:L 
            B = basis(;L, N)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E

        # P
        Es = []
        for p in [1, -1] 
            B = basis(;L, p)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # Z
        Es = []
        for z in [1, -1] 
            B = basis(;L, z)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # k 
        Es = []
        for k in 0:L-1
            B = basis(;L, k)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E
    end
end

@testset "2-sym-vals" begin
    for L in 4:10 
        E = trans_inv_operator(mat, 4, L) |> Hermitian |> eigvals
        
        # N + P 
        Es = []
        for N in 0:L, p in [1, -1]
            B = basis(;L, N, p)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # N + k
        Es = []
        for N in 0:L, k in 0:L-1
            B = basis(;L, N, k)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # N + Z 

        # P + Z
        Es = []
        for p in [1, -1], z in [1, -1]
            B = basis(;L, p, z)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # k + Z
        Es = []
        for k in 0:L-1, z in [1, -1]
            B = basis(;L, k, z)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # k + P
        
    end
end


@testset "Half-k" begin
    for L in 4:2:10, k in [0, L÷2]
        B = basis(;L, k); iszero(size(B, 1)) && continue  
        E = trans_inv_operator(mat, 4, B) |> Hermitian |> eigvals

        # k + P 
        Es = []
        for p in [1, -1]
            B = basis(;L, k, p)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # k + P + Z
        Es = []
        for p in [1, -1], z in [1, -1]
            B = basis(;L, k, p, z)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # k + P + N
        Es = []
        for N in 0:L, p in [1, -1]
            B = basis(;L, N, k, p)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 
    end
end

@testset "Half-filling" begin
    for L in 4:2:10
        N = L÷2
        B = basis(;L, N); iszero(size(B, 1)) && continue  
        E = trans_inv_operator(mat, 4, B) |> Hermitian |> eigvals

        # N + Z 
        Es = []
        for z in [1, -1]
            B = basis(;L, N, z)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # N + Z + P
        Es = []
        for z in [1, -1], p in [1, -1]
            B = basis(;L, N, p, z)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 

        # N + Z + k
        Es = []
        for k in 0:L-1, z in [1, -1]
            B = basis(;L, N, k, z)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 
    end
end

@testset "Symmetric" begin
    for L in 4:2:12, k in [0, L÷2]
        N = L÷2
        
        B = basis(;L, N, k); iszero(size(B, 1)) && continue  
        E = trans_inv_operator(mat, 4, B) |> Hermitian |> eigvals

        # N + k + P + Z
        Es = []
        for p in [1, -1], z in [1, -1]
            B = basis(;L, N, k, p, z)
            add_value!(Es, B)
        end
        @test sort(Es) ≈ E 
    end
end

nothing

Test Summary: | Pass  Total  Time
1-sym-vals    |   28     28  4.0s
Test Summary: | Pass  Total  Time
2-sym-vals    |   28     28  1.2s
Test Summary: | Pass  Total  Time
Half-k        |   24     24  0.2s
Test Summary: | Pass  Total  Time
Half-filling  |   12     12  0.0s
Test Summary: | Pass  Total  Time
Symmetric     |   10     10  0.4s


## Entanglement Test

In [3]:
function add_value!(M::AbstractMatrix, E, S, B)
    iszero(size(B, 1)) && return 
    e, v = trans_inv_operator(M, round(Int, log(2, size(M, 1))), B) |> Hermitian |> eigen
    append!(E, e)
    inds = 1:length(B)÷2 
    s = [ent_S(v[:, i], inds, B) for i in axes(v, 2)]
    append!(S, s)
end

function add_value!(Ms::Vector, E, S, B)
    iszero(size(B, 1)) && return 
    n = round(Int, log(2, size(Ms[1], 1)))
    L = length(B)
    inds = [mod.(i-1:i+n-2, L) .+ 1 for i in 1:L]
    e, v = operator(Ms, inds, B) |> Hermitian |> eigen
    append!(E, e)
    s = [ent_S(v[:, i], 1:L÷2, B) for i in axes(v, 2)]
    append!(S, s)
end

@testset "Relax k" begin
    for L in 4:8
        M = randn(ComplexF64, 8, 8) |> Hermitian |> Array  

        B = basis(;L); iszero(size(B, 1)) && continue  
        E, V = trans_inv_operator(M, 3, B) |> Hermitian |> eigen
        S = [ent_S(V[:, i], 1:L÷2, B) for i in axes(V, 2)] |> sort
        
        Es, Ss = [], []
        for k in 0:L-1
            B = basis(;L, k)
            add_value!(M, Es, Ss, B)
        end
        @test sort(Es) ≈ E
        @test norm(sort(Ss) - S) < 1e-7
    end
end



@testset "Relax N" begin
    a = 1.0 
    b = randn()
    c = randn()
    d = randn()
    e = randn()
    f = 5

    mat = spin(
        (a, "xx11"), (a, "yy11"), 
        (b, "x1x1"), (b, "y1y1"), 
        (c, "x11x"), (c, "y11y"), 
        (d, "zz11"),
        (e, "z1z1"), 
        (f, "z111")
    )
    for L in 4:8
        mats = [randn()*mat for i in 1:L]
        inds = [mod.(i-1:i+2, L) .+ 1 for i in 1:L]

        B = basis(;L); iszero(size(B, 1)) && continue  
        E, V = operator(mats, inds, B) |> Hermitian |> eigen
        S = [ent_S(V[:, i], 1:L÷2, B) for i in axes(V, 2)] |> sort
        
        Es, Ss = [], []
        for N in 0:L
            B = basis(;L, N)
            add_value!(mats, Es, Ss, B)
        end
        @test sort(Es) ≈ E
        @test norm(sort(Ss) - S) < 1e-7
        
    end
end

@testset "k -> N" begin
    a = 1.0 
    b = randn()
    c = randn()
    d = randn()
    e = randn()
    f = 5

    mat = spin(
        (a, "xx11"), (a, "yy11"), 
        (b, "x1x1"), (b, "y1y1"), 
        (c, "x11x"), (c, "y11y"), 
        (d, "zz11"),
        (e, "z1z1"), 
        (f, "z111")
    )

    for L in 4:8, k in 0:L-1
        B = basis(;L, k); iszero(size(B, 1)) && continue  
        E, V = trans_inv_operator(mat, 4, B) |> Hermitian |> eigen
        S = [ent_S(V[:, i], 1:L÷2, B) for i in axes(V, 2)] |> sort
        
        Es, Ss = [], []
        for N in 0:L
            B = basis(;L, N, k)
            add_value!(mat, Es, Ss, B)
        end
        @test sort(Es) ≈ E
        @test norm(sort(Ss) - S) < 1e-7
        
    end
end

@testset "N k -> P Z" begin
    for L in 4:2:12, k in [0, L÷2]
        N = L÷2

        # N + k + P + Z
        B = basis(;L, N, k); iszero(size(B, 1)) && continue  
        E, V = trans_inv_operator(mat, 4, B) |> Hermitian |> eigen
        S = [ent_S(V[:, i], 1:L÷2, B) for i in axes(V, 2)] |> sort
        
        Es, Ss = [], []
        for p in [1, -1], z in [1, -1]
            B = basis(;L, N, k, p, z)
            add_value!(mat, Es, Ss, B)
        end
        @test sort(Es) ≈ E
        @test sort(Ss) ≈ S
    end
end

nothing

Test Summary: | Pass  Total  Time
Relax k       |   10     10  4.3s
Test Summary: | Pass  Total  Time
Relax N       |   10     10  0.8s
Test Summary: | Pass  Total  Time
k -> N        |   60     60  0.2s
Test Summary: | Pass  Total  Time
N k -> P Z    |   20     20  0.4s


## Benchmark

In [4]:
using BenchmarkTools
@benchmark TranslationalBasis(L=6, k=1)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   25.647 μs … 859.257 ms  ┊ GC (min … max):  0.00% … 99.95%
 Time  (median):     186.594 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   308.167 μs ±   8.606 ms  ┊ GC (mean ± σ):  27.87% ±  1.00%

                  ▄█▅                                            
  ▂▂▃▂▂▂▂▁▁▁▁▂▃▃▃▇████▄▃▃▂▂▂▂▃▃▃▂▂▂▁▁▁▁▁▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  25.6 μs          Histogram: frequency by time          570 μs <

 Memory estimate: 265.97 KiB, allocs estimate: 176.

In [5]:
@benchmark basis(L=6, k=1)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  23.421 μs … 101.262 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     24.482 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.960 μs ±   3.807 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▆▇██▇▅▂▁                                                    ▂
  ██████████▆▇▆▇▇▇▆▇▇▇▇█▇▇▅▅▄▄▅▄▁▃▄▁▄▃▄▅▃▄▃▅▅▅▅▅▄▄▃▃▅▄▄▃▅▄▄▃▄▃ █
  23.4 μs       Histogram: log(frequency) by time      39.4 μs <

 Memory estimate: 19.56 KiB, allocs estimate: 167.

In [1]:
include("../EDKit.jl")
using Main.EDKit
B = basis(L=20, k=1)
size(B)

(52377, 52377)

In [14]:
@time B0 = TranslationalBasis(L=20, k=1)
size(B0)

  0.039026 seconds (202 allocations: 2.582 MiB)


(52377, 52377)